In [5]:
/media/data/users/fderop/software/STAR/bin/Linux_x86_64/STAR\
    --runThreadN 16 \
    --runMode genomeGenerate \
    --genomeDir /media/data/users/fderop/00000000_res/mm10_STAR_premrna/ \
    --genomeFastaFiles /media/data/users/fderop/00000000_res/mm10_STAR_premrna/genome.fa \
    --sjdbGTFfile /media/data/users/fderop/00000000_res/mm10_STAR_premrna/genes.gtf \
    --sjdbOverhang 100

	/media/data/users/fderop/software/STAR/bin/Linux_x86_64/STAR --runThreadN 24 --runMode genomeGenerate --genomeDir /media/data/users/fderop/00000000_res/mm10_STAR/ --genomeFastaFiles /media/data/users/fderop/00000000_res/mm10_STAR/Mus_musculus.GRCm38.dna.primary_assembly.fa --sjdbGTFfile /media/data/users/fderop/00000000_res/mm10_STAR/gencode.vM23.primary_assembly.annotation.gtf --sjdbOverhang 100
	STAR version: 2.7.8a_2021-04-27   compiled: 2021-04-27T10:22:15-0400 vega:/home/dobin/data/STAR/STARcode/STAR.master/source
Apr 28 14:30:17 ..... started STAR run
Apr 28 14:30:17 ... starting to generate Genome files
Apr 28 14:31:08 ..... processing annotations GTF
Apr 28 14:31:35 ... starting to sort Suffix Array. This may take a long time...
Apr 28 14:31:53 ... sorting Suffix Array chunks and saving them to disk...
Apr 28 14:45:28 ... loading chunks from disk, packing SA...
Apr 28 14:46:41 ... finished generating suffix array
Apr 28 14:46:41 ... generating Suffix Array index
Apr 28 14:51:3

# Fastp QC
Perform fastp to get a html report of the sequencing. We can see in the fastp htmls that the barcode was sequenced corrrectly.

In [49]:
module load fastp
for fastq in fastq/*R2_001.fastq.gz
do
    echo $fastq
    html=${fastq%_001.fastq.gz}fastp_report.html
    fastp -i $fastq -I $fastq -h $html -w 8 &
done

fastq/HYR__525037__20210225_R16_S1_R2_001.fastq.gz
[1] 8299
fastq/HYR__81e131__20210225_R74_S2_R2_001.fastq.gz
[2] 8300


In [ ]:
module load fastp
for fastq in fastq/*R1_001.fastq.gz
do
    echo $fastq
    html=${fastq%_001.fastq.gz}fastp_report.html
    fastp -i $fastq -I $fastq -h $html -w 8 &
done

The hydrop-rna library looks like this:

```
               Index 1: i7 read 10 bp           Read 1: cDNA read 86 bp  

             <- HyDrop_CustSeq_Short_i7_Read            <- NXT               
|-5’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC_UMI]-[polyA]-[cDNA]-[NXT]-[i5]-[P5]-3’
  3’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC_UMI]-[polyT]-[cDNA]-[NXT]-[i5]-[P5]-5’-|  
                HyDrop_CustSeq_Short ->                    NXT ->    
                
                   Read 2: BC 60 bp            Index 2: i5 read 10 bp       

```

The barcode read is ultimately structured like this:

```
-[BC1]-[AD1]-[BC2]-[AD2]-[BC3]-[UMI]
-[10b]-[10b]-[10b]-[10b]-[10b]-[8bp]
```

With the lengths of each part under the part. The adapters inbetween the 3 sub-barcodes are identical for all reads.

we need to use STARsolo to map AND barcode demultiplex these samples. The star solo manual is excellent and explains all parameters used. Please change `--soloCellFilter CellRanger2.2 5000 0.99 10 \` to the correct number of expected cells.

In [7]:
star_solo_hydrop_rna_mapping_unique_cellfilter () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    # 3 barcode lists:
    local whitelist_part1_filename='/media/data/users/fderop/00000000_barcodes/20200227_HyDrop-ATAC_96x96x96/20200227_bc1.txt';
    local whitelist_part2_filename='/media/data/users/fderop/00000000_barcodes/20200227_HyDrop-ATAC_96x96x96/20200227_bc2.txt';
    local whitelist_part3_filename='/media/data/users/fderop/00000000_barcodes/20200227_HyDrop-ATAC_96x96x96/20200227_bc3.txt';
    star_reference_dir='/media/data/users/fderop/00000000_res/mm10_STAR_premrna/';
    sjdbgtf='/media/data/users/fderop/00000000_res/mm10_STAR_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /media/data/users/fderop/software/STAR/bin/Linux_x86_64/STAR \
        --runThreadN 6 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist "${whitelist_part1_filename}" "${whitelist_part2_filename}" "${whitelist_part3_filename}" \
        --soloType CB_UMI_Complex \
        --soloCBposition 0_0_0_9 0_20_0_29 0_40_0_49 \
        --soloUMIposition 0_50_0_57 \
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --soloCBmatchWLtype 1MM \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull Velocyto
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [ ]:
dir=demultiplexed
mkdir $dir
for sample in fastq/*R2_001.fastq.gz
do
    NAME=${sample%_R2_001.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter \
        ${NAME}_R1_001.fastq.gz \
        ${NAME}_R2_001.fastq.gz \
        $dir/$OUT.bam
done

Now, we can get a summary of the statistics below:

In [3]:
for sample in /lustre1/project/stg_00002/lcb/fderop/data/20210428_hydrop-rna_mouse_cortex_10k/demultiplexed/HYR__666332__20210412_mouse_cortex_sa_S1.Solo.out/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

lustre1/project/stg_00002/lcb/fderop/data/20210428_hydrop-rna_mouse_cortex_10k/demultiplexed/HYR__666332__20210412_mouse_cortex_sa_S1.Solo.out/Gene/Summary.csv
Number of Reads,242071788
Reads With Valid Barcodes,0.83327
Sequencing Saturation,0.551516
Q30 Bases in CB+UMI,0.774848
Q30 Bases in RNA read,0.950998
Reads Mapped to Genome: Unique+Multiple,0.859084
Reads Mapped to Genome: Unique,0.859084
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.56905
Reads Mapped to Transcriptome: Unique Genes,0.560225
Estimated Number of Cells,5232
Reads in Cells Mapped to Unique Genes,47987067
Fraction of Reads in Cells,0.353848
Mean Reads per Cell,9171
Median Reads per Cell,7941
UMIs in Cells,21186594
Mean UMI per Cell,4049
Median UMI per Cell,3514
Mean Genes per Cell,1778
Median Genes per Cell,1693
Total Genes Detected,21719



In [4]:
for sample in /lustre1/project/stg_00002/lcb/fderop/data/20210428_hydrop-rna_mouse_cortex_10k/demultiplexed/HYR__c47998_e6244b__20210412_mouse_cortex_sbmerged_S4.Solo.out/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

lustre1/project/stg_00002/lcb/fderop/data/20210428_hydrop-rna_mouse_cortex_10k/demultiplexed/HYR__c47998_e6244b__20210412_mouse_cortex_sbmerged_S4.Solo.out/Gene/Summary.csv
Number of Reads,269012338
Reads With Valid Barcodes,0.827878
Sequencing Saturation,0.659605
Q30 Bases in CB+UMI,0.780043
Q30 Bases in RNA read,0.954299
Reads Mapped to Genome: Unique+Multiple,0.859785
Reads Mapped to Genome: Unique,0.859785
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.559121
Reads Mapped to Transcriptome: Unique Genes,0.550297
Estimated Number of Cells,4459
Reads in Cells Mapped to Unique Genes,50489518
Fraction of Reads in Cells,0.341061
Mean Reads per Cell,11323
Median Reads per Cell,9793
UMIs in Cells,16851724
Mean UMI per Cell,3779
Median UMI per Cell,3265
Mean Genes per Cell,1718
Median Genes per Cell,1624
Total Genes Detected,21483



There will be a STARsolo output which contains both raw matrix (all barcodes) and filtered matrix (only includes "cells", which are determined using the same filter as cellranger uses)